In [ ]:
from datetime import datetime, timedelta
import json
from pytz import timezone as tz
import requests
from functools import lru_cache


In [ ]:
BASE_URL: str = "https://statsapi.mlb.com/api/v1"
CURR_SEASON: str = str(datetime.now().year)


def getData(endpoint: str, **kwargs) -> dict:
    queryParams: str = (
        f"&{'&'.join(f'{k}={v}' for k, v in kwargs.items())}" if kwargs else ""
    )
    url = f"{BASE_URL}/{endpoint}?sportId=1&season={CURR_SEASON}{queryParams}"
    return requests.get(url).json()


In [ ]:
@lru_cache(maxsize=1)
def _teamsIndex() -> dict:
    index = {}
    for team in getData("teams").get("teams", []):
        aliases = {
            team.get("name"),
            team.get("teamName"),
            team.get("abbreviation"),
            team.get("shortName"),
            team.get("locationName"),
        }
        teamId = team.get("id")
        for alias in aliases:
            if alias:
                index[alias.lower()] = teamId
    return index


def getTeamId(teamName: str) -> int | None:
    return _teamsIndex().get(teamName.lower())


In [ ]:
@lru_cache(maxsize=1)
def _playersIndex() -> dict:
    players = (
        requests.get("https://statsapi.mlb.com/api/v1/sports/1/players")
        .json()
        .get("people", [])
    )
    index = {}
    for player in players:
        aliases = {
            player.get("firstLastName"),
            player.get("lastFirstName"),
        }
        playerId = player.get("id")
        for alias in aliases:
            if alias:
                index[alias.lower()] = playerId
    return index


def getPlayerId(playerName: str) -> int | None:
    return _playersIndex().get(playerName.lower())


In [ ]:
def printSample(endpoint: str, **kwargs) -> None:
    sampleTitle = kwargs.get(
        "sampleTitle", f"MLB {endpoint.capitalize()} Response Sample"
    )
    sampleSize = kwargs.get("size", None)
    print(f"\n{'='*len(sampleTitle)}\n{sampleTitle}\n{'='*len(sampleTitle)}\n")
    response = getData(endpoint, **kwargs).get(endpoint, [])
    sampleSize = kwargs.get(
        "sampleSize", len(response) if not sampleSize else sampleSize
    )
    data = response[0] if len(response) == 1 else response[0:sampleSize]
    print(json.dumps(data, indent=2))


In [ ]:
printSample("seasons")



MLB Seasons Response Sample

{
  "seasonId": "2026",
  "hasWildcard": true,
  "preSeasonStartDate": "2026-01-01",
  "preSeasonEndDate": "2026-02-19",
  "seasonStartDate": "2026-02-20",
  "springStartDate": "2026-02-20",
  "springEndDate": "2026-03-24",
  "regularSeasonStartDate": "2026-03-25",
  "lastDate1stHalf": "2026-07-14",
  "allStarDate": "2026-07-14",
  "firstDate2ndHalf": "2026-07-19",
  "regularSeasonEndDate": "2026-09-27",
  "postSeasonStartDate": "2026-09-28",
  "postSeasonEndDate": "2026-10-31",
  "seasonEndDate": "2026-10-31",
  "offseasonStartDate": "2026-11-01",
  "offSeasonEndDate": "2026-12-31",
  "seasonLevelGamedayType": "P",
  "gameLevelGamedayType": "P",
  "qualifierPlateAppearances": 3.1,
  "qualifierOutsPitched": 3.0
}


In [46]:
printSample("leagues", sampleSize=2)



MLB Leagues Response Sample

[
  {
    "id": 103,
    "name": "American League",
    "link": "/api/v1/league/103",
    "abbreviation": "AL",
    "nameShort": "American",
    "seasonState": "preseason",
    "hasWildCard": true,
    "hasSplitSeason": false,
    "numGames": 162,
    "hasPlayoffPoints": false,
    "numTeams": 15,
    "numWildcardTeams": 3,
    "seasonDateInfo": {
      "seasonId": "2026",
      "preSeasonStartDate": "2026-01-01",
      "preSeasonEndDate": "2026-02-19",
      "seasonStartDate": "2026-02-20",
      "springStartDate": "2026-02-20",
      "springEndDate": "2026-03-24",
      "regularSeasonStartDate": "2026-03-25",
      "lastDate1stHalf": "2026-07-14",
      "allStarDate": "2026-07-14",
      "firstDate2ndHalf": "2026-07-19",
      "regularSeasonEndDate": "2026-09-27",
      "postSeasonStartDate": "2026-09-28",
      "postSeasonEndDate": "2026-10-31",
      "seasonEndDate": "2026-10-31",
      "offseasonStartDate": "2026-11-01",
      "offSeasonEndDate": "202

In [ ]:
printSample("divisions", sampleSize=3)



MLB Divisions Response Sample

[
  {
    "id": 200,
    "name": "American League West",
    "season": "2026",
    "nameShort": "AL West",
    "link": "/api/v1/divisions/200",
    "abbreviation": "ALW",
    "league": {
      "id": 103,
      "link": "/api/v1/league/103"
    },
    "sport": {
      "id": 1,
      "link": "/api/v1/sports/1"
    },
    "hasWildcard": false,
    "sortOrder": 24,
    "numPlayoffTeams": 1,
    "active": true
  },
  {
    "id": 201,
    "name": "American League East",
    "season": "2026",
    "nameShort": "AL East",
    "link": "/api/v1/divisions/201",
    "abbreviation": "ALE",
    "league": {
      "id": 103,
      "link": "/api/v1/league/103"
    },
    "sport": {
      "id": 1,
      "link": "/api/v1/sports/1"
    },
    "hasWildcard": false,
    "sortOrder": 22,
    "numPlayoffTeams": 1,
    "active": true
  },
  {
    "id": 202,
    "name": "American League Central",
    "season": "2026",
    "nameShort": "AL Central",
    "link": "/api/v1/divisions/20

In [ ]:
printSample("teams", sampleSize=4)



MLB Teams Response Sample

[
  {
    "springLeague": {
      "id": 114,
      "name": "Cactus League",
      "link": "/api/v1/league/114",
      "abbreviation": "CL"
    },
    "allStarStatus": "N",
    "id": 133,
    "name": "Athletics",
    "link": "/api/v1/teams/133",
    "season": 2026,
    "venue": {
      "id": 2529,
      "name": "Sutter Health Park",
      "link": "/api/v1/venues/2529"
    },
    "springVenue": {
      "id": 2507,
      "link": "/api/v1/venues/2507"
    },
    "teamCode": "ath",
    "fileCode": "ath",
    "abbreviation": "ATH",
    "teamName": "Athletics",
    "locationName": "Sacramento",
    "firstYearOfPlay": "1901",
    "league": {
      "id": 103,
      "name": "American League",
      "link": "/api/v1/league/103"
    },
    "division": {
      "id": 200,
      "name": "American League West",
      "link": "/api/v1/divisions/200"
    },
    "sport": {
      "id": 1,
      "link": "/api/v1/sports/1",
      "name": "Major League Baseball"
    },
    "shortN

In [ ]:
printSample(
    "leagues", leagueIds=104, sampleTitle="MLB Specific League (NL) Response Sample"
)



MLB Specific League (NL) Response Sample

{
  "id": 104,
  "name": "National League",
  "link": "/api/v1/league/104",
  "abbreviation": "NL",
  "nameShort": "National",
  "seasonState": "preseason",
  "hasWildCard": true,
  "hasSplitSeason": false,
  "numGames": 162,
  "hasPlayoffPoints": false,
  "numTeams": 15,
  "numWildcardTeams": 3,
  "seasonDateInfo": {
    "seasonId": "2026",
    "preSeasonStartDate": "2026-01-01",
    "preSeasonEndDate": "2026-02-19",
    "seasonStartDate": "2026-02-20",
    "springStartDate": "2026-02-20",
    "springEndDate": "2026-03-24",
    "regularSeasonStartDate": "2026-03-25",
    "lastDate1stHalf": "2026-07-14",
    "allStarDate": "2026-07-14",
    "firstDate2ndHalf": "2026-07-19",
    "regularSeasonEndDate": "2026-09-27",
    "postSeasonStartDate": "2026-09-28",
    "postSeasonEndDate": "2026-10-31",
    "seasonEndDate": "2026-10-31",
    "offseasonStartDate": "2026-11-01",
    "offSeasonEndDate": "2026-12-31",
    "seasonLevelGamedayType": "P",
    

In [ ]:
printSample(
    "divisions",
    divisionId=204,
    sampleTitle="MLB Specific Division (NL) Response Sample",
)



MLB Specific Division (NL) Response Sample

{
  "id": 204,
  "name": "National League East",
  "season": "2026",
  "nameShort": "NL East",
  "link": "/api/v1/divisions/204",
  "abbreviation": "NLE",
  "league": {
    "id": 104,
    "link": "/api/v1/league/104"
  },
  "sport": {
    "id": 1,
    "link": "/api/v1/sports/1"
  },
  "hasWildcard": false,
  "sortOrder": 32,
  "numPlayoffTeams": 1,
  "active": true
}


In [ ]:
printSample(
    "teams", teamId=144, sampleTitle="MLB Specific Team (Braves) Response Sample"
)



MLB Specific Team (Braves) Response Sample

{
  "springLeague": {
    "id": 115,
    "name": "Grapefruit League",
    "link": "/api/v1/league/115",
    "abbreviation": "GL"
  },
  "allStarStatus": "N",
  "id": 144,
  "name": "Atlanta Braves",
  "link": "/api/v1/teams/144",
  "season": 2026,
  "venue": {
    "id": 4705,
    "name": "Truist Park",
    "link": "/api/v1/venues/4705"
  },
  "springVenue": {
    "id": 5380,
    "link": "/api/v1/venues/5380"
  },
  "teamCode": "atl",
  "fileCode": "atl",
  "abbreviation": "ATL",
  "teamName": "Braves",
  "locationName": "Atlanta",
  "firstYearOfPlay": "1871",
  "league": {
    "id": 104,
    "name": "National League",
    "link": "/api/v1/league/104"
  },
  "division": {
    "id": 204,
    "name": "National League East",
    "link": "/api/v1/divisions/204"
  },
  "sport": {
    "id": 1,
    "link": "/api/v1/sports/1",
    "name": "Major League Baseball"
  },
  "shortName": "Atlanta",
  "franchiseName": "Atlanta",
  "clubName": "Braves",
  "a

In [ ]:
def printTeamTotalSeasonWins(teamName: str, season: int = None) -> None:
    teamId = getTeamId(teamName)
    year = season or CURR_SEASON
    isCurrSeason = year == CURR_SEASON

    gameDates = getData(
        "schedule", teamId=teamId, startDate=f"{year}-01-01", endDate=f"{year}-12-31"
    ).get("dates", [])

    totalWins = 0
    for games in gameDates:
        for game in games.get("games", []):
            teams = game.get("teams", {})
            homeTeamResult = {
                "id": teams.get("home", {}).get("team", {}).get("id"),
                "isWinner": teams.get("home", {}).get("isWinner", False),
            }
            awayTeamResult = {
                "id": teams.get("away", {}).get("team", {}).get("id"),
                "isWinner": teams.get("away", {}).get("isWinner", False),
            }
            winningTeamId = (
                homeTeamResult["id"]
                if homeTeamResult["isWinner"]
                else awayTeamResult["id"] if awayTeamResult["isWinner"] else None
            )

            if winningTeamId == teamId:
                totalWins += 1

    print(
        f"The {teamName}{' have won ' if isCurrSeason else ' won '}{totalWins} games {'this season' if isCurrSeason else f'in {year}'}."
    )


In [44]:
printTeamTotalSeasonWins("Atlanta Braves", season=2021)
printTeamTotalSeasonWins("Atlanta Braves", season=2022)
printTeamTotalSeasonWins("Atlanta Braves", season=2023)
printTeamTotalSeasonWins("Atlanta Braves", season=2024)
printTeamTotalSeasonWins("Atlanta Braves", season=2025)
printTeamTotalSeasonWins("Atlanta Braves")


The Atlanta Braves won 114 games in 2021.
The Atlanta Braves won 110 games in 2022.
The Atlanta Braves won 124 games in 2023.
The Atlanta Braves won 101 games in 2024.
The Atlanta Braves won 90 games in 2025.
The Atlanta Braves have won 0 games this season.


In [ ]:
def getLogoUrl(teamId: int | None = None, teamName: str | None = None, **kwargs) -> str:
    teamId = teamId if teamId else getTeamId(teamName) if teamName else None
    logoType = kwargs.get("logoType", "cap")
    logoShade = kwargs.get("logoShade", "light")
    return f"https://www.mlbstatic.com/team-logos/team-{logoType}-on-{logoShade}/{teamId}.svg"


def getHeadshotUrl(playerId: int | None = None, playerName: str | None = None) -> str:
    playerId = playerId if playerId else getPlayerId(playerName) if playerName else None
    return f"https://img.mlbstatic.com/mlb-photos/image/upload/d_people:generic:headshot:67:current.png/w_213,q_auto:best/v1/people/{playerId}/headshot/67/current"
